# Multi-Agents with AgentTool

Let's setup a multi-agent system using agents as tools so that a single agent can route between them based
on the task (to use them as specialized experts).

In [1]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.tools import AgentTool
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
# Set up model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key=os.environ.get("OPENAI_API_KEY"),
)
model_client = OpenAIChatCompletionClient(model="gpt-4.1")

data_engineer_agent = AssistantAgent(
    "data_engineer",
    model_client=model_client,
    system_message="You are a data engineering expert specializing in ETL pipelines and data infrastructure.",
    description="A data engineering expert assistant.",
    model_client_stream=True,
)
data_engineer_tool = AgentTool(data_engineer_agent, return_value_as_last_message=True)

cloud_architect_agent = AssistantAgent(
    "cloud_architect",
    model_client=model_client,
    system_message="You are a cloud architecture expert with deep knowledge of cloud deployment and scaling strategies.",
    description="A cloud architecture expert assistant.",
    model_client_stream=True,
)
cloud_architect_tool = AgentTool(cloud_architect_agent, return_value_as_last_message=True)

agent = AssistantAgent(
    "assistant",
    system_message="You are a general assistant. Use expert tools when needed.",
    model_client=model_client,
    model_client_stream=True,
    tools=[data_engineer_tool, cloud_architect_tool],
    max_tool_iterations=10,
)
await Console(agent.run_stream(task="How would you design a scalable ETL pipeline for real-time analytics on AWS?"))